New logic for landmark similarity

In [2]:
%pip install requests pandas time tqdm

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import requests
import time

df = pd.read_csv("output_cleaned.csv")  
# Sample the first 10 rows for testing
df_sample = df.head(10).copy()


API_KEY = "API_KEY"  # replace this  

df_sample["geocoded_latitude"] = None
df_sample["geocoded_longitude"] = None


def geocode_address(address, api_key):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": address,
        "key": api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        result = response.json()
        if result["status"] == "OK":
            location = result["results"][0]["geometry"]["location"]
            return location["lat"], location["lng"]
    return None, None


for i in range(len(df_sample)):
    addr = df_sample.loc[i, "address"]
    lat, lng = geocode_address(addr, API_KEY)
    df_sample.loc[i, "geocoded_latitude"] = lat
    df_sample.loc[i, "geocoded_longitude"] = lng
    print(f"{i+1}/10 geocoded: {addr} -> ({lat}, {lng})")
    time.sleep(1)  # Sleep to avoid hitting API rate limits

df_sample.to_csv("geocoded_test_output.csv", index=False)

print("Done! Output saved to geocoded_test_output.csv")


1/10 geocoded:  atur park basant garden chembur mumbai  400071 -> (19.0514337, 72.9045968)
2/10 geocoded: tilak road opp bikaji ghatkopare 400077 -> (19.0829313, 72.90709509999999)
3/10 geocoded: station road vikhroliwest station road 400083 -> (19.1108928, 72.9270139)
4/10 geocoded: oppshreyas cinema lbsmarg ghatkoparw 400082 -> (19.1284994, 72.9252816)
5/10 geocoded: nr bhaji mandi kurlaw  400070 -> (19.065709, 72.8758792)
6/10 geocoded: suleman khatri building  299f kurla west  kurla 400070 -> (19.0684302, 72.87566319999999)
7/10 geocoded: shop no 03 vikas commercial centre chembur east near basant cinema 400074 -> (19.086383, 72.8889773)
8/10 geocoded: shop no30dipak center safed pul saki naka mumbai 400072 -> (19.0985731, 72.8850052)
9/10 geocoded: navketan building n g aacharya marg chembur east  400071 -> (19.0617585, 72.9009066)
10/10 geocoded: chita camp mankhurd east 400088 -> (19.0472598, 72.9285969)
Done! Output saved to geocoded_test_output.csv


In [ ]:
import pandas as pd
import requests
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import time


df = pd.read_csv("geocoded_test_output.csv")  # Replace with actual file name from previous step
df = df.head(10)  # Only 10 rows for testing

API_KEY = "API_KEY"  # replace this

def get_landmarks(lat, lon, radius=50):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lon}",
        "radius": radius,
        "key": API_KEY
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        return [place["name"] for place in data.get("results", [])]
    except Exception as e:
        print(f"Error fetching landmarks for ({lat}, {lon}): {e}")
        return []

tqdm.pandas()
df["orig_landmarks"] = df.progress_apply(
    lambda row: get_landmarks(row["latitude"], row["longitude"]), axis=1
)
df["geocoded_landmarks"] = df.progress_apply(
    lambda row: get_landmarks(row["geocoded_latitude"], row["geocoded_longitude"]), axis=1
)

model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

def compute_similarity(orig_list, geocoded_list):
    if not orig_list or not geocoded_list:
        return 0.0
    try:
        orig_text = ", ".join(orig_list)
        geo_text = ", ".join(geocoded_list)

        emb1 = model.encode(orig_text, convert_to_tensor=True)
        emb2 = model.encode(geo_text, convert_to_tensor=True)

        return util.pytorch_cos_sim(emb1, emb2).item()
    except Exception as e:
        print(f"Similarity error: {e}")
        return 0.0

df["landmark_similarity"] = df.progress_apply(
    lambda row: compute_similarity(row["orig_landmarks"], row["geocoded_landmarks"]),
    axis=1
)

high_sim_matches = df[df["landmark_similarity"] > 0.75]
print(f"✔️ High similarity landmark pairs: {len(high_sim_matches)} out of {len(df)}")

df.to_csv("new_landmark_similarity_results_sample10.csv", index=False)


100%|██████████| 10/10 [00:03<00:00,  3.12it/s]

✔️ High similarity landmark pairs: 3 out of 10
